In [ ]:
from fastai.vision.all import *
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool

import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib

%matplotlib inline

In [ ]:
ROOT = '../input/hpa-single-cell-image-classification/'
cell_mask_dir = '../input/hpa-mask/test/hpa_cell_mask'
train_or_test = 'test'

In [ ]:
def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode('ascii')

In [ ]:
def decodeToBinaryMask(rleCodedStr, imWidth, imHeight):
    uncodedStr = base64.b64decode(rleCodedStr)
    uncompressedStr = zlib.decompress(uncodedStr,wbits = zlib.MAX_WBITS)   
    detection ={
        'size': [imWidth, imHeight],
        'counts': uncompressedStr
    }
    detlist = []
    detlist.append(detection)
    mask = coco_mask.decode(detlist)
    binaryMask = mask.astype('bool') 
    return binaryMask

In [ ]:
def get_cropped_cell(img, msk):
    bmask = msk.astype(int)[...,None]
    masked_img = img * bmask
    true_points = np.argwhere(bmask)
    top_left = true_points.min(axis=0)
    bottom_right = true_points.max(axis=0)
    cropped_arr = masked_img[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1]
    return cropped_arr

In [ ]:
def read_img(image_id, color, train_or_test='train', image_size=None):
    filename = f'{ROOT}/{train_or_test}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    return img

In [ ]:
def pred_one_img(image_id, class_num=19, thres=0.4):
    cell_mask = np.load(f'{cell_mask_dir}/{image_id}.npz')['arr_0']
    red = read_img(image_id, "red", train_or_test, None)
    
    PS = []
    for cell in range(1, np.max(cell_mask) + 1):
        bmask = cell_mask == cell
        encoder = encode_binary_mask(bmask)

        fname = f'{image_id}_{cell}'
        
        pred_score = pred_dict[fname]
        
        for i in range(class_num):
            PS.append(' '.join([str(i), str(pred_score[i]), encoder]))
            
    PS = ' '.join(PS)
    
    return PS

In [ ]:
pred_img_lst = np.load('../result/imgs.npy')
pred_img_lst = ['_'.join(item.split('_')[:2]) for item in pred_img_lst]
pred_probs_lst = np.load('../result/probs.npy')

pred_dict = dict(zip(pred_img_lst, pred_probs_lst))

In [ ]:
sb = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv'); sb.head()

In [ ]:
for idx in tqdm(range(len(sb))):
    
    ID = sb.loc[idx]['ID']
    PS = pred_one_img(ID)
    
    sb.loc[idx, 'PredictionString'] = PS

In [ ]:
sb.to_csv('../result/submission.csv', index=None)